# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [ ]:
df = pd.read_csv('/content/dataset.csv')
places = df['SystemCodeNumber'].unique()
places

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

In [5]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [6]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [7]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    SystemCodeNumber : str
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location


In [8]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [9]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Step 2: Making a simple pricing function

In [ ]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time
    .windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .groupby(pw.this.SystemCodeNumber)
    .reduce(
        SystemCodeNumber = pw.reducers.argmax(pw.this.t, pw.this.SystemCodeNumber),
        occupancy = pw.reducers.argmax(pw.this.t, pw.this.Occupancy),
        capacity = pw.reducers.argmax(pw.this.t, pw.this.Capacity),
        t = pw.reducers.argmax(pw.this.t, pw.this.t),
        day = pw.reducers.argmax(pw.this.t, pw.this.day),
    )
)

delta_window


In [ ]:
pw.io.csv.write(delta_window, '/content/output.csv')

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

In [ ]:
df = pd.read_csv('/content/output.csv')
df['t'] = pd.to_datetime(df['t'])
df['price'] = 10
df

In [ ]:
for place in places:
   mask = df['SystemCodeNumber'] == place
   df_1 = df[mask]
   data = df_1.copy().reset_index(drop=True)


#Update prices row-by-row using previous row
   for i in range(1, len(data)):
    prev_price = data.loc[i - 1, 'price']
    prev_occ = data.loc[i-1, 'occupancy']
    prev_cap = data.loc[i-1, 'capacity']
    occ = data.loc[i, 'occupancy']
    cap = data.loc[i, 'capacity']
    if (occ/cap) < (prev_occ/prev_cap):
      alpha = -0.2
    else:
      alpha = 0.2

    data.loc[i, 'price'] = prev_price + alpha * (occ / cap)



   df.loc[mask, 'price'] = data['price'].values


df


# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [ ]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.io import curdoc
from bokeh.plotting import ColumnDataSource, figure, output_file, show


# print(df)
output_file("dashboard.html")

curdoc().theme = 'dark_minimal'


def plot_price_fluctuations(df, place):

   curdoc().theme = 'dark_minimal'
   p = figure(title=place ,width=1000, height=400, x_axis_type="datetime")
   p.line(df.t, df.price)
   p.scatter(df.t, df.price, fill_color="red", size=5)
   return p

panels = {}
for idx, lot in enumerate(places):
    lot_data = df[(df.SystemCodeNumber == lot)]
    viz = plot_price_fluctuations(lot_data, lot)
    panels[lot] = viz

dashboard = pn.Tabs(*[(str(lot), panel) for lot, panel in panels.items()])

dashboard.servable()  # Use .show() for Jupyter/Colab, or .servable() for Panel server


# for place in places:
#    plot_price_fluctuations(df[df['SystemCodeNumber'] == place], place)
